In [ ]:
import openai
from typing import List

### Vanilla OpenAI

In [ ]:
client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello!"}]
)
response.choices[0].message.content

In [ ]:

prompt_template = "Tell me a short joke about {topic}"
client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("a parrot")

### With LangChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

#from langchain_mistralai.chat_models import ChatMistralAI


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
#model = ChatMistralAI(api_key=api_key)
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

chain.invoke("a parrot")

### What is this "|"?

In [4]:
class CustomLCEL:
    def __init__(self, value):
        self.value = value

    def __or__(self, other):
        if callable(other):
            return CustomLCEL(other(self.value))
        else:
            raise ValueError("Right operand must be callable")

    def result(self):
        return self.value

def to_uppercase(s):
    return s.upper()

def add_exclamation(s):
    return s + "!"

def reverse_string(s):
    return s[::-1]

In [5]:
custom_chain = (
    CustomLCEL("hello world")
    | to_uppercase
    | add_exclamation
    | reverse_string
)

result = custom_chain.result()
print(result)

!DLROW OLLEH


### Runnable Interface (Stream, Invoke, Batch)

In [ ]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

In [ ]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="", flush=True)

In [ ]:
await chain.ainvoke({"topic": "bears"})

In [ ]:
await chain.abatch([{"topic": "bears"}])